In [6]:
using QuantumOptics
using Combinatorics
using Einsum

In [7]:
#= """
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `sp_matrix::Matrix`: Hopping phases matrix from any model.
"""
function get_sp_op(sp_basis, sp_matrix)
    
    H = SparseOperator(sp_basis)

    N, = size(sp_matrix)
    
    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end  =#

get_sp_op

In [8]:
"""
Compute the first eigen-states in a given number.

#### Arguments
- `sp_op::get_sp_op`: Single-particle operator.
- `cut_off::Int`: Limit value of eigen-state index.
"""
function get_subset_states(sp_op, sub_range)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[sub_range]
    
    return states
end

get_subset_states

In [9]:
#= """
Compute sub-space basis, projection and comlex conjugate of projection operator.

#### Arguments
- `states::get_sub_states`: Eigen-states of the sub-space.
- `basis::NLevelBasis`: Single-particle basis.
"""
function get_projector_op(basis, proj_basis)
    
    subspace_proj = SubspaceBasis(basis, proj_basis)
    P = projector(subspace_proj, basis)
    Pt = dagger(P)
    
    return subspace_proj, P, Pt
end =#

get_projector_op

In [10]:
#= """
Compute the corresponding operator in the sub-space.

#### Arguments
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end =#

get_subspace_op

In [11]:
#= """
Compute the many-body operator for boson particles from single-particle operator.

#### Arguments
- `mb_basis`: Many-body basis.
- `sp_op::Operator`: Single-particle operator.
"""
function get_mb_op(mb_basis, sp_op)
    
    mb_op = SparseOperator(mb_basis)

    # site number
    N = size(sp_op)[1]
    
    for i in 1:N
        for j in 1:N
            mb_op += sp_op.data[i,j] * transition(mb_basis, i, j)
        end
    end
    
    return mb_op
end =#

get_mb_op

In [12]:
function get_H_mb(param_dict)
    
    H1_m = param_dict["model_H1"](param_dict)    
    H1 = get_sp_op(param_dict["basis"], H1_m)
    
    # param_dict0 = copy(param_dict)
    # param_dict0["θθ"]=[ 0, 0 ]
    # H10_m = param_dict["model_H1"](param_dict0)    
    # H10 = get_sp_op(param_dict["basis"], H10_m)
    
    # sub space
    if "sub_range" in keys(param_dict)
        
        sub_states = get_subset_states(H1, param_dict["sub_range"]);
        #subspace_proj, P, Pt = get_projector_op(param_dict["basis"], sub_states)
        #param_dict["subspace_proj"] = subspace_proj
        param_dict["projection"] = get_projector_op(param_dict["basis"], sub_states)
        
        # DÜZENLENDİ!
        #subspace_proj, P, Pt = param_dict["projection"]
        #H1_proj = get_subspace_op(H1, P, Pt)
        H1_proj = get_subspace_op(H1, param_dict["projection"][2], param_dict["projection"][3])
        
        basis_mb = get_basis_mb(param_dict)
        param_dict["dimH_mb"] = length(basis_mb)
        H_mb = get_mb_op(param_dict["basis_mb"], H1_proj)
        H_mb = (H_mb'+H_mb)/2
        
    # full space
    else
        basis_mb = get_basis_mb(param_dict)
        param_dict["dimH_mb"] = length(basis_mb)
        H_mb = get_mb_op(param_dict["basis_mb"], H1)
    end
    return H_mb
end

get_H_mb (generic function with 1 method)

In [13]:
function get_basis_mb(param_dict)
    
    # sub space
    if "sub_range" in keys(param_dict)
        states = bosonstates(param_dict["projection"][1], param_dict["PN"])
        basis_mb = ManyBodyBasis(param_dict["projection"][1], states)
        param_dict["basis_mb"] = basis_mb
        
    # full space
    else
        states = bosonstates(param_dict["basis"], param_dict["PN"])
        basis_mb = ManyBodyBasis(param_dict["basis"], states)
        param_dict["basis_mb"] = basis_mb
    end
    return basis_mb
end

get_basis_mb (generic function with 1 method)

In [14]:
function get_basis_mb_int(param_dict)
    
    # sub space
    if "sub_range" in keys(param_dict)
        states = bosonstates(param_dict["projection"][1], param_dict["PN_int"])
        basis_mb = ManyBodyBasis(param_dict["projection"][1], states)
        param_dict["basis_mb"] = basis_mb
        
    # full space
    else
        states = bosonstates(param_dict["basis"], param_dict["PN_int"])
        basis_mb = ManyBodyBasis(param_dict["basis"], states)
        param_dict["basis_mb"] = basis_mb
    end
    return basis_mb
end

get_basis_mb_int (generic function with 1 method)

In [15]:
# BUNU SADECE INTERACTION YAZARKEN KULLANIYORUM

function get_H_mb_2(param_dict)
    
    H1_m = param_dict["model_H1"](param_dict)
    
    H1 = get_sp_op(param_dict["basis"], H1_m)
    
    sub_states = get_subset_states(H1, param_dict["sub_range"])
    subspace_proj, P, Pt = get_projector_op(param_dict["basis"], sub_states)
    param_dict["projection"] = subspace_proj, P, Pt
    
    # DÜZENLENDİ!
    #H1_proj = get_subspace_op(H1, P, Pt)
    H1_proj = get_subspace_op(H1, param_dict["projection"][2], param_dict["projection"][3])

    basis_mb = get_basis_mb_int(param_dict)
    param_dict["basis_mb"] = basis_mb
    H_mb = get_mb_op(param_dict["basis_mb"], H1_proj)
    H_mb = (H_mb'+H_mb)/2
    
    #mb_states_chern = param_dict["mb_states_chern"] #???

    # DÜZENLENDİ!
    #mb_states = bosonstates(subspace_proj, param_dict["PN"])
    #İhtiyacım yok
    #mb_states = bosonstates(param_dict["projection"][1], param_dict["PN"])
    #ihtiyacım yok
    #param_dict["mb_states"] = mb_states
    #basis_mb_chern = param_dict["basis_mb_chern"] #???

    # BASIS TRANSFORMATION
    # DÜZENLENDİ!
    #UPM = param_dict["projection"][2]*param_dict["projection_chern"][2]'

    #şimdilik ihtiyacım yok
    #UPM = param_dict["projection"][2]*param_dict["projection_chern"][3]
    #UP = get_mb_transform(basis_mb_chern,basis_mb,mb_states_chern,mb_states,UPM)
    
    return H_mb, P, Pt
end

get_H_mb_2 (generic function with 1 method)

In [16]:
# ????
function get_mb_transform(mb_basis,mb_basisx,mb_states,mb_statesx,P)
    state_list, norms = get_state_list(mb_states)
    state_listx, normsx = get_state_list(mb_statesx)
    UP = Matrix{ComplexF64}(undef, length(state_list), length(state_listx))
    for (i_mb, mb_state_list) in enumerate(state_list)
        Ni = norms[i_mb]
    for (j_mb, mb_state_listx) in enumerate(state_listx)
        Nj = normsx[j_mb]
        UUij = 0
        #for state in permutations(mb_state_list)
        for statex in permutations(mb_state_listx)
            Uij = 1
            for (i1,j1) in zip( mb_state_list, statex)
                   Uij *= P.data[i1,j1]
                   #println(i1, " ", j1, " ", P.data[i1,j1])
            end
            #println(mb_state_list," ", statex)
            UUij += Uij
        end
        #end
        UP[i_mb, j_mb] = Ni*Nj*UUij
        #println(i_mb, " ", j_mb, " : ", UP[i_mb, j_mb], " ", Ni," ", Nj," ", mb_state_list," ", mb_state_listx)
    end
    end
    return Operator(mb_basis,UP)
end

get_mb_transform (generic function with 1 method)

In [17]:
function get_state_list(mbstates)
    state_list = []
    norms = []
    for mb_state in mb_states
        state_idx = []
        for (iocc, occ) in enumerate(mb_state)
            state_idx = vcat(state_idx,repeat([iocc],occ))
        end
        # println(state_idx)
        push!(state_list,state_idx)
        N2 = prod( filter(x->x≠0, mb_state) )
        push!(norms, 1/sqrt(N2))
    end
    return state_list, norms
end

get_state_list (generic function with 1 method)

In [18]:
function get_Int_Part(N, MB_Basis, U)
    
    IT = SparseOperator(MB_Basis)
    for m in 1:N
        IT += U/2 * number(MB_Basis, m) * ( number(MB_Basis, m) - identityoperator(MB_Basis) ) 
    end
    
    return IT
end

get_Int_Part (generic function with 1 method)

In [ ]:
function get_Finite_U(param_dict)
    H_mb = get_H_mb(param_dict)
    H_Int = get_Int_Part(N, basis_mb, param_dict["U"])
    H_total =  H_mb + H_Int
    return H_total, H_Int
end

In [1]:
#= function get_Bosonic_MB_Basis_for_cut_off_space(N,PN)
   
    NBasis = NLevelBasis(N)
    NStates = bosonstates(NBasis, PN)
    
    NBasis_MB = ManyBodyBasis(NBasis, NStates)
    
    return NBasis_MB, NBasis
end =#

get_Bosonic_MB_Basis_for_cut_off_space (generic function with 1 method)

In [22]:
function get_Hub(param_dict)
    U = param_dict["U"]
    proj1, P1, P1t = param_dict["projection"]
    P = P1.data
    Pt = P1t.data
    basis_mb = param_dict["basis_mb"]
    @einsum coefficient[k,j] := P[j,i] * P[k,i] * Pt[i,k] * Pt[i,j]

    Vint_mb = SparseOperator(basis_mb)

    D1_proj = length(proj1)
    for k in 1:D1_proj
        nk = number(basis_mb, k)
        for j in 1:D1_proj
            nj = number(basis_mb, j)
            if k == j
                Vint_mb += U/2*coefficient[k,k]*nk*(nk-identityoperator(basis_mb))
            else
                Vint_mb += U*coefficient[k,j]*nk*nj
            end
        end
    end
    return Vint_mb
end

get_Hub (generic function with 1 method)

In [24]:
function get_Hub_2(param_dict)
    U = param_dict["U"]
    proj1, P1, P1t = param_dict["projection"]
    P = P1.data
    Pt = P1t.data
    basis_mb = param_dict["basis_mb"]
    @einsum coefficient[k,j] := P[j,i] * P[k,i] * Pt[i,k] * Pt[i,j]
    @einsum coefficient2[j] := P[j,i] * Pt[i,j]

    Vint_mb = SparseOperator(basis_mb)

    D1_proj = length(proj1)
    for k in 1:D1_proj
        nk = number(basis_mb, k)
        for j in 1:D1_proj
            nj = number(basis_mb, j)
            if k == j
                Vint_mb += U/2*(coefficient[k,k]*nk*nk-coefficient2[k]*nk)
            else
                Vint_mb += U/2*coefficient[k,j]*(2*nk*nj+nj)
            end
        end
    end
    return Vint_mb
end

get_Hub_2 (generic function with 1 method)

In [ ]:
function get_Hub_3(param_dict)
    U = param_dict["U"]
    proj1, P1, P1t = param_dict["projection"]
    P = P1.data
    Pt = P1t.data
    basis_mb = param_dict["basis_mb"]
    @einsum coefficient[k,j] := P[j,i] * P[k,i] * Pt[i,k] * Pt[i,j]
    
    D1_proj = length(proj1)
    
    for k in 1:D1_proj
        for j in 1:D1_proj
            Vint_mb = U/2*many
        end
    end
end

In [5]:
function Hubbard_Interaction(P, Pt, cut_mb_basis, cut_off, U)
    
    P1 = P.data
    P1t = Pt.data

    @einsum coefficient[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(cut_mb_basis)
        
    for k in 1:cut_off
        for l in 1:cut_off
            for m in 1:cut_off
                for n in 1:cut_off
                    a1t = create(cut_mb_basis, k)
                    a2t = create(cut_mb_basis, l)
                    a2  = destroy(cut_mb_basis, m)      
                    a1  = destroy(cut_mb_basis, n)      
                    Vint_mb_cut += U/2*coefficient[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    
    return Vint_mb_cut
end

Hubbard_Interaction (generic function with 1 method)

In [ ]:
function get_energies(pn, E, UU, basis)
    
    PN_Energies = Array{Float64}(undef, length(E), 2)
    for i in 1:length(E)
        PN_Energies[i] = round(expect(number(basis), UU[i])) 
        PN_Energies[i,2] = E[i] 
    end
    
    df = DataFrame(PN_Energies, :auto)
    df = filter(row -> (row.x1 == pn),  df)
    
    return df
end

In [ ]:
function proj_MB_Int(param_dict)
    
    H_kin_sub, PP, PPt = get_H_mb_2(param_dict)
    basis_cut_mb, basis_cut_sp = get_Bosonic_MB_Basis_for_cut_off_space(cut_off, param_dict["PN_int"])
    H_Int = Hubbard_Interaction(PP, PPt, basis_cut_mb, cut_off, param_dict["U"])
    
    H_Kin = SparseOperator(basis_cut_mb)
    H_Kin.data = H_kin_sub.data
    
    H_Total_Int = H_Kin + H_Int
    return H_Total_Int, basis_cut_mb
end